# Round 3 verification 

In [3]:
import os
import sys
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/')) # Assuming the DummyDataGenerator is defined in a module

import numpy as np
import pandas as pd
from  round3_op_implementation import ProjectAllocator
from model.DummyDataGenerator import DummyDataGenerator 
from model.VotingRules import VotingRules

# Constants for allocation
QUORUM = 17
MIN_AMOUNT = 1500
TOTAL_AMOUNT = 30000000

# Generate dummy data
generator = DummyDataGenerator(
    num_rows=100,  # Number of rows (voters) to generate
    max_project_in_ballot=5,  # Maximum projects a voter can vote for
    max_votes=3,  # Maximum votes a voter can cast
    start_date="2023-01-01",  # Start date for random timestamps
    end_date="2023-12-31"  # End date for random timestamps
)

# Generate the DataFrame
df = generator.generate_dummy_data()

# Expand the 'Votes' column and format the DataFrame for ProjectAllocator
df_votes_expanded = df.explode('Votes').reset_index(drop=True)
df_votes_expanded = pd.concat(
    [df_votes_expanded.drop(['Votes'], axis=1), df_votes_expanded['Votes'].apply(pd.Series)],
    axis=1
)
df_votes_expanded = df_votes_expanded.rename(columns={
    "projectId": "project_id",
    "amount": "amount",
    "Address": "voter_address"
})
df_votes_expanded["amount"] = df_votes_expanded["amount"].astype(float)  # Ensure amount is a float

# Instantiate the allocator and calculate initial and scaled allocations
allocator = ProjectAllocator(TOTAL_AMOUNT, MIN_AMOUNT, QUORUM)
initial_allocation = allocator.calculate_initial_allocation(df_votes_expanded)
print('Initial_Allocation')
print(initial_allocation)

scaled_allocation = allocator.scale_allocations(initial_allocation, "median_amount")
print('Scaled_Allocation')
print(scaled_allocation)

# Now let's use your VotingRules class to calculate the allocation using the median rule
voting_matrix = df_votes_expanded.pivot_table(
    index='voter_address', columns='project_id', values='amount', fill_value=0
).values


num_voters = voting_matrix.shape[0]
total_op_tokens = TOTAL_AMOUNT  # Equivalent to total funds in this context


voting_rules = VotingRules()
median_allocation = voting_rules.round3_median(voting_matrix, total_op_tokens, num_voters)

comparison_df = pd.DataFrame({
    'Allocation (ProjectAllocator)': scaled_allocation['scaled_amount'].values,
    'Allocation (Our Implemenation)': median_allocation
})
print('Comparison_Allocation')
comparison_df.head()

2024-08-10 02:59:28 INFO | Check - If all are eligible: False
2024-08-10 02:59:28 INFO | Check - Original Amount Eligible: 11201914.0
2024-08-10 02:59:28 INFO | Check - Scale Factor: 2.67811375805956
2024-08-10 02:59:28 INFO | Check - New Amount Eligible: 29999999.999999996


Initial_Allocation
            votes_count  median_amount  is_eligible
project_id                                         
proj0                10      1656956.5        False
proj1                18      2635983.5         True
proj2                17      2426158.0         True
proj3                17      2522421.0         True
proj4                24      1960395.0         True
Scaled_Allocation
            votes_count  median_amount  is_eligible  scaled_amount
project_id                                                        
proj0                10      1656956.5        False   4.437518e+06
proj1                18      2635983.5         True   7.059464e+06
proj2                17      2426158.0         True   6.497527e+06
proj3                17      2522421.0         True   6.755330e+06
proj4                24      1960395.0         True   5.250161e+06
Comparison_Allocation


,Allocation (ProjectAllocator),Allocation (Our Implemenation)
0,4.437518e+06,4.437518e+06
1,7.059464e+06,7.059464e+06
2,6.497527e+06,6.497527e+06
3,6.755330e+06,6.755330e+06
4,5.250161e+06,5.250161e+06
